In [1]:
import pandas as pd
import numpy as np
import keras
from keras.layers import concatenate,Input,Dense, GlobalAveragePooling1D, Embedding, Dropout
from keras.layers import LSTM, Flatten, Activation, Conv1D,SpatialDropout1D,GRU,GlobalMaxPooling1D
from keras.models import Sequential,Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from keras.callbacks import Callback
import re
import nltk
from nltk.corpus import wordnet

import nltk
import string
import re
from tqdm import tqdm
from string import punctuation
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

/Users/dsg281/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('/Users/dsg281/Downloads/Toxic_Comments_Iden/train.csv',nrows=2000)
test = pd.read_csv('/Users/dsg281/Downloads/Toxic_Comments_Iden/test.csv',nrows=2000)
submission = pd.read_csv('/Users/dsg281/Downloads/Toxic_Comments_Iden/sample_submission.csv')


In [3]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
WORDS = {}
index = [1]

In [5]:
def get_coefs(word, *arr): 
    WORDS[word] = index[0]
    index[0] = index[0]+1
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open('/Users/dsg281/Downloads/Embedding_Vectors/FastText/crawl-300d-2M.vec'))

In [6]:
######## Spelling correction
def P(word): 
    return - WORDS.get(word, 0)

def correction(word): 
    return max(candidates(word), key=P)

def candidates(word): 
    return (known([word]) or known(edits1(word)) or [word])

def known(words): 
    return set(w for w in words if w in WORDS)

def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


In [7]:
cols  = list(train.columns.values)
cols.remove('id')
cols.remove('comment_text')
print(cols)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [8]:
######## Correct misspelling in both train and test data
def corrected_words(row):
    comment = row['comment_text']
    result =  ' '
    for w in nltk.word_tokenize(comment):
        if len(w)>1 and not wordnet.synsets(w):
            result += correction(w) + ' '
        else:
            result += w + ' '
    
    return result

In [9]:
train['comment_text'] = train.apply(corrected_words, axis=1)
test['comment_text'] = test.apply(corrected_words, axis=1)

In [10]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,Daww ! He matches this background colour I 'm...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man , I 'm really not trying to edit war ...",0,0,0,0,0,0
3,0001b41b1c6bb37e,`` More I ca not make any real suggestions on...,0,0,0,0,0,0
4,0001d958c54c6e35,"You , sir , are my hero . Any chance you reme...",0,0,0,0,0,0
5,00025465d4725e87,"`` Congratulations from me as well , use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington articl...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word nonsense ' was offensive to...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contr...,0,0,0,0,0,0


In [11]:
def text_to_wordlist(text, remove_stop_words=True, stem_words=False):
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [12]:
train['comment_text'] = [text_to_wordlist(text) for text in train['comment_text']]
test['comment_text'] = [text_to_wordlist(text) for text in test['comment_text']]

In [13]:
train['comment_text'] = train.apply(corrected_words, axis=1)
test['comment_text'] = test.apply(corrected_words, axis=1)

In [14]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why edits made username Hardcore ...,0,0,0,0,0,0
1,000103f0d9cfb60f,Daww He matches background colour I seemingly...,0,0,0,0,0,0
2,000113f07ec002fd,Hey man I really trying edit war It guy const...,0,0,0,0,0,0
3,0001b41b1c6bb37e,More I ca make real suggestions improvement I...,0,0,0,0,0,0
4,0001d958c54c6e35,You sir hero Any chance remember page,0,0,0,0,0,0
5,00025465d4725e87,Congratulations well use tools well · talk,0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism Matt Shirvington article rever...,0,0,0,0,0,0
8,00037261f536c51d,Sorry word nonsense offensive Anyway I intend...,0,0,0,0,0,0
9,00040093b2687caa,alignment subject contrary DuLithgow,0,0,0,0,0,0


In [22]:
######## text tokenization
max_features = 15000
max_len=100

In [23]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['comment_text'])
tokenizer.fit_on_texts(test['comment_text'])

In [24]:
def process_text(data):
    sequences = tokenizer.texts_to_sequences(data)
    return pad_sequences(sequences=sequences, maxlen=max_len)

In [25]:
train_data = process_text(train['comment_text'])
test_data = process_text(test['comment_text'])

In [26]:
input_dim = np.max([np.max(train_data), np.max(test_data)]) + 1
embedding_dims = 300
print(input_dim)

15000


In [27]:
######## Load pretrained embedding data
word_index = tokenizer.word_index

In [28]:
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embedding_dims))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [29]:
######## Build Model [Embedding and CNN]
def clf_model():
    seq = Input(shape=(max_len,), name='seq')
    l = Embedding(max_features, embedding_dims, weights=[embedding_matrix])(seq)
    l = SpatialDropout1D(0.4)(l)
    c2 = Conv1D(embedding_dims,2, activation='relu')(l)
    c3 = Conv1D(embedding_dims,3, activation='relu')(l)
    c4 = Conv1D(embedding_dims,4, activation='relu')(l)
    c5 = Conv1D(embedding_dims,5, activation='relu')(l)
    max_pool2 = GlobalMaxPooling1D()(c2)
    max_pool3 = GlobalMaxPooling1D()(c3)
    max_pool4 = GlobalMaxPooling1D()(c4)
    max_pool5 = GlobalMaxPooling1D()(c5)
    conc = concatenate([max_pool2,max_pool3,max_pool4,max_pool5])
    conc = Dense(36)(conc)
    output = Dense(6, activation='sigmoid', name='output')(conc)
    model = Model(inputs=[seq], outputs=output)
    model.compile(loss='binary_crossentropy',optimizer='adam')
    return model

In [30]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            print("starting")
            print("starting")
x_train, x_test, y_train, y_test = train_test_split(train_data, train[cols], test_size = 0.05, random_state = 144)
RocAuc = RocAucEvaluation(validation_data=(x_test, y_test), interval=1)
text_clf = clf_model()


In [ ]:
submission = pd.DataFrame()
submission['id'] = test.id

In [ ]:
text_clf.fit(x_train, y_train, batch_size=50, epochs=2, validation_data=(x_test, y_test), callbacks=[RocAuc], verbose=0)
pred = text_clf.predict(test_data)
i=0
for col in cols:
    submission[col] = pred[:,i]
    i+=1
